In [7]:
pip install openai

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 12.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 20.8 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Carrega variáveis do .env
load_dotenv()

# Recupera chave da API
api_key = os.getenv("OPENAI_API_KEY")

# Cria cliente OpenAI com a chave
client = OpenAI(api_key=api_key)

In [16]:
def gerar_insight_profissoes():
    prompt = """
    Liste as 10 profissões emergentes mais promissoras em Inteligência Artificial para os próximos 3 anos. 
    Para cada profissão, inclua:
    - Nome do cargo
    - Breve descrição
    - Principais habilidades exigidas
    - Tecnologias mais utilizadas
    - Setores com maior demanda

    Responda no formato de tabela Markdown.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=1000
    )

    return response.choices[0].message.content

In [18]:
resposta = gerar_insight_profissoes()
print(resposta)

| Cargo | Descrição | Principais Habilidades | Tecnologias Mais Utilizadas | Setores com Maior Demanda |
|-------|-----------|-----------------------|---------------------------|--------------------------|
| Cientista de Dados em IA | Responsável por desenvolver modelos de machine learning e algoritmos de IA para análise de dados. | Conhecimento em estatística, programação, machine learning e big data. | Python, R, TensorFlow, scikit-learn | Tecnologia, Finanças, Saúde |
| Engenheiro de Machine Learning | Encarregado de desenvolver e implementar algoritmos de machine learning em sistemas e aplicações. | Conhecimento em programação, matemática, estatística e machine learning. | Python, TensorFlow, PyTorch, scikit-learn | Tecnologia, Finanças, Marketing |
| Arquiteto de IA | Responsável por projetar e implementar soluções de inteligência artificial em sistemas complexos. | Conhecimento em IA, programação, arquitetura de software e segurança da informação. | Python, TensorFlow, Microsoft 

In [21]:
import pandas as pd
from io import StringIO

# Converte a string Markdown para um DataFrame
df = pd.read_csv(StringIO(resposta), sep="|", engine="python", skipinitialspace=True)

# Remove colunas vazias que o Markdown pode gerar nas bordas
df = df.loc[:, df.columns.str.strip() != '']

# Limpa nomes de colunas
df.columns = df.columns.str.strip()

# Limpa conteúdo das células
df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

# Visualiza resultado
print(df.head())

# Salva em CSV (poderia ser .json ou banco também)
df.to_csv('../data/insights_openai.csv', index=False)

   Unnamed: 0                           Cargo  \
0         NaN                         -------   
1         NaN        Cientista de Dados em IA   
2         NaN  Engenheiro de Machine Learning   
3         NaN                 Arquiteto de IA   
4         NaN         Analista de Dados em IA   

                                           Descrição  \
0                                        -----------   
1  Responsável por desenvolver modelos de machine...   
2  Encarregado de desenvolver e implementar algor...   
3  Responsável por projetar e implementar soluçõe...   
4  Encarregado de analisar dados e desenvolver in...   

                              Principais Habilidades  \
0                            -----------------------   
1  Conhecimento em estatística, programação, mach...   
2  Conhecimento em programação, matemática, estat...   
3  Conhecimento em IA, programação, arquitetura d...   
4  Conhecimento em estatística, análise de dados,...   

                       Tecnolog

In [36]:
import sys
sys.path.append('../src')  # se estiver um nível acima da pasta src
from transform_openai import markdown_to_dataframe
import importlib

importlib.reload(sys.modules['transform_openai'])
# Usa a função para converter a resposta em DataFrame
df_insights = markdown_to_dataframe(resposta)
df_insights.to_csv('../data/insights_openai.csv', index=False)

# Exibe os dados
df_insights.head()

,Cargo,Descrição,Principais Habilidades,Tecnologias Mais Utilizadas,Setores com Maior Demanda
0,Cientista de Dados em IA,Responsável por desenvolver modelos de machine...,"Conhecimento em estatística, programação, mach...","Python, R, TensorFlow, scikit-learn","Tecnologia, Finanças, Saúde"
1,Engenheiro de Machine Learning,Encarregado de desenvolver e implementar algor...,"Conhecimento em programação, matemática, estat...","Python, TensorFlow, PyTorch, scikit-learn","Tecnologia, Finanças, Marketing"
2,Arquiteto de IA,Responsável por projetar e implementar soluçõe...,"Conhecimento em IA, programação, arquitetura d...","Python, TensorFlow, Microsoft Azure, IBM Watson","Tecnologia, Saúde, Automotivo"
3,Analista de Dados em IA,Encarregado de analisar dados e desenvolver in...,"Conhecimento em estatística, análise de dados,...","Python, R, SQL, TensorFlow","Finanças, Marketing, Varejo"
4,Especialista em NLP,Responsável por desenvolver e implementar solu...,"Conhecimento em linguística, machine learning,...","Python, NLTK, spaCy, GPT-3","Tecnologia, Saúde, Educação"
